## MNIST Classification using PyTorch 

In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets

### Loading the Data

In [2]:
train = datasets.MNIST('', train=True,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))


test = datasets.MNIST('', train=False,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

100.1%

Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw


113.5%

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw


100.4%

Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw


180.4%

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw
Processing...
Done!


In [4]:
trainset = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [21]:
img_shape = train.data[0].shape[0]
img_shape

28

In [25]:
num_classes = len(train.classes)
num_classes

10

### Building the Neural Network

In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [52]:
class Network(nn.Module):
    def __init__(self, img_shape=28, num_classes=10):
        super().__init__()
        
        self.img_shape = img_shape
        self.dense1_op = 64
        self.dense2_op = 64
        self.dense3_op = 64
        self.num_classes = num_classes
        
        
        self.dense1 = nn.Linear(img_shape * img_shape, self.dense1_op)
        self.dense2 = nn.Linear(self.dense1_op, self.dense2_op)
        self.dense3 = nn.Linear(self.dense2_op, self.dense3_op)
        self.dense4 = nn.Linear(self.dense3_op, self.num_classes)
        
    
    ## forward pass
    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        x = self.dense4(x)
        
        return F.log_softmax(x, dim=1)

In [53]:
net = Network(img_shape=img_shape, num_classes=num_classes)

In [54]:
net

Network(
  (dense1): Linear(in_features=784, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=64, bias=True)
  (dense3): Linear(in_features=64, out_features=64, bias=True)
  (dense4): Linear(in_features=64, out_features=10, bias=True)
)

### Training the Network

In [55]:
from torch.optim import Adam

In [56]:
optimizer = Adam(net.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [57]:
epochs = 10

In [64]:
from tqdm import tqdm

In [65]:
for epoch in tqdm(range(epochs)):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1, img_shape*img_shape))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print('Loss: ', loss)

 10%|████████▎                                                                          | 1/10 [00:17<02:36, 17.42s/it]

Loss:  tensor(0.0107, grad_fn=<NllLossBackward>)


 20%|████████████████▌                                                                  | 2/10 [00:35<02:20, 17.58s/it]

Loss:  tensor(0.0315, grad_fn=<NllLossBackward>)


 30%|████████████████████████▉                                                          | 3/10 [00:53<02:04, 17.76s/it]

...

Loss:  tensor(0.0035, grad_fn=<NllLossBackward>)


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:40<00:17, 17.86s/it]

Loss:  tensor(0.0001, grad_fn=<NllLossBackward>)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:58<00:00, 17.80s/it]

Loss:  tensor(0.0031, grad_fn=<NllLossBackward>)


### Accuracy

In [66]:
def accuracy(dataset, net=net):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in dataset:
            X, y = data
            output = net(X.view(-1, img_shape*img_shape))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total += 1
    print('Accuracy: ', round(correct/total, 3) * 100)

#### Train Accuracy

In [67]:
accuracy(trainset)

Accuracy:  99.7


#### Test Accuracy

In [68]:
accuracy(testset)

Accuracy:  97.6


#### Credits to SentDex on Youtube - https://www.youtube.com/channel/UCfzlCWGWYyIQ0aLC5w48gBQ